# Example 1: Get protein information, run BLAST, and dump the results into a database


In [1]:
%reload_ext autoreload
%autoreload 2
from pyEED.core import ProteinInfo

## Query NCBI

The pyEED library is centered around the `ProteinSequence` object, which integrates available information on protein sequence, corresponding nucleotide sequence, as well as regions and sites within the sequences. The `ProteinSequence` can be initialized directly with a protein sequence accession number.

In [2]:
aldolase = ProteinInfo.from_ncbi("NP_001287541.1")
print(aldolase)

ProteinInfo
├── id = proteininfo0
├── source_id = NP_001287541.1
├── name = aldolase 1, isoform M
├── sequence = MTTYFNYPSKELQDELREIAQKIVAPGKGILAADESGPTMGKRLQDIGVENTEDNRRAYRQLLFSTDPKLAENISGVILFHETLYQKADDGTPFAEILKKKGIILGIKVDKGVVPLFGSEDEVTTQGLDDLAARCAQYKKDGCDFAKWRCVLKIGKNTPSYQSILENANVLARYASICQSQRIVPIVEPEVLPDGDHDLDRAQKVTETVLAAVYKALSDHHVYLEGTLLKPNMVTAGQSAKKNTPEEIALATVQALRRTVPAAVTGVTFLSGGQSEEEATVNLSAINNVPLIRPWALTFSYGRALQASVLRAWAGKKENIAAGQNELLKRAKANGDAAQGKYVAGSAGAGSGSLFVANHAY
├── organism
│   └── Organism
│       ├── id = organism0
│       ├── name = Drosophila melanogaster
│       ├── taxonomy_id = taxon:7227
│       ├── domain = Eukaryota
│       ├── kingdom = Metazoa
│       ├── phylum = Arthropoda
│       ├── tax_class = Insecta
│       ├── order = Diptera
│       ├── family = Drosophilidae
│       ├── genus = Drosophila
│       └── species = melanogaster
├── regions
│   └── 0
│       └── ProteinRegion
│           ├── id = proteinregion0
│           ├── name = Glycolytic
│           ├── 

In [3]:
# Get the corresponding DNA sequence
aldolase_dna = aldolase.get_dna()
print(aldolase_dna)

DNAInfo
├── id = dnainfo0
├── name = Drosophila melanogaster aldolase 1, transcript variant M (Ald1), mRNA
├── sequence = AGTCGGTTGCCCAACACCGTCGGCGACGACCTCCAAACAACCTTTTGGGGTCTCGCTCGGCATGTCTCGCTTATTTCGCTTTGGAATTTCACGTTGACGCCGCAGTCAGCGGTAGCAGAGGCATCGGCAGCGGCAGTAGCAGCGACACGGGCCGAAAATAAAAGCGTTAACCGCTCTCCTCCAGTGCAGCAGCAGCAGCGGACCCAGCCAGTAGCCAGCAGCTAATCACCACATCCCAGATTCAGTTTCCAGTTCGAACTACACTCGAATCTCAAAAATGACGACCTACTTCAACTACCCCAGCAAGGAGCTGCAGGATGAGCTGCGCGAAATCGCCCAGAAAATCGTTGCCCCCGGCAAGGGAATCCTCGCCGCCGATGAGTCCGGCCCAACCATGGGCAAGCGTCTGCAGGACATCGGCGTGGAGAACACCGAGGACAACCGCCGTGCCTACCGTCAGCTGTTGTTCAGCACTGACCCCAAGCTGGCCGAGAACATCTCTGGAGTGATCCTGTTCCACGAGACCCTCTACCAGAAGGCCGATGATGGCACCCCCTTCGCCGAGATCCTGAAGAAGAAGGGAATCATTCTGGGCATCAAGGTCGACAAGGGTGTTGTCCCACTGTTCGGCTCTGAGGATGAGGTCACCACCCAGGGTCTGGATGACCTGGCCGCCCGTTGCGCCCAGTACAAGAAGGACGGTTGCGACTTCGCCAAGTGGCGTTGCGTCCTGAAGATCGGCAAGAACACCCCATCCTACCAGTCGATCCTGGAGAACGCCAATGTCCTGGCCCGCTACGCCTCCATCTGCCAGTCGCAGCGCATCGTCCCAATTGTGGAGCCCGAGGTTCTGCCCGATGGCGATCACGATCTGGACC

## BLAST search

In [23]:
blast_results = aldolase.pblast(n_hits=99, e_value=0.0021)

🏃🏼‍♀️ Running PBLAST
├── name: aldolase 1, isoform M
├── organism: Drosophila melanogaster
├── e-value: 0.0021
└── max hits: 99


Fetching protein sequences: 99it [00:20,  4.87it/s]


## Storing `ProteinSequence`s in a PostgreSQL database



In [8]:
from sdrdm_database import DBConnector

### Setting up a local MySQL database

First, a local MySQL database needs to be setup. Therefore, we run a docker container with a MySQL database. 
If docker is not installed on your system, please follow the instructions on the [docker website](https://docs.docker.com/get-docker/).


In case this notebook is run on a macOS system with a M1 chip, the following command needs to be run in the terminal first:

>```bash
>export DOCKER_DEFAULT_PLATFORM=linux/amd64
>```

Next, navigate to the directory where this notebook is located and run the following command to start the docker container:

>```bash
>docker compose up -d
>```

### Delete contianers

>```    
>docker rm -vf $(docker ps -aq)
>docker rmi -f $(docker images -aq)
>```

### Connect to the PostgreSQL database

In [9]:
import toml

# Establish a connection to the database
db = DBConnector(**toml.load(open("./env.toml")))

🎉 Connected                                                                                        


### Create tables for `ProteinInfo`

In [10]:
db.create_tables(
    model=ProteinInfo,
    markdown_path="/Users/max/Documents/GitHub/pyeed/specifications/data_model.md",
)


🚀 Creating tables for data model ProteinInfo
│
├── Model 'ProteinInfo' already registered. Skipping.
├── Table 'ProteinInfo'. Already exists in database. Skipping.
├── Table 'ProteinInfo_coding_sequence_ref'. Already exists in database. Skipping.
├── Table 'DNARegion_spans'. Already exists in database. Skipping.
├── Table 'ProteinInfo_sites'. Already exists in database. Skipping.
├── Table 'Site_positions'. Already exists in database. Skipping.
├── Table 'ProteinInfo_regions'. Already exists in database. Skipping.
├── Table 'ProteinRegion_spans'. Already exists in database. Skipping.
├── Table 'ProteinInfo_organism'. Already exists in database. Skipping.
│
╰── 🎉 Created all tables for data model ProteinInfo



In [11]:
# See all created table names
db.connection.list_tables()

['DNARegion_spans',
 'ProteinInfo',
 'ProteinInfo_coding_sequence_ref',
 'ProteinInfo_organism',
 'ProteinInfo_regions',
 'ProteinInfo_sites',
 'ProteinRegion_spans',
 'Site_positions',
 '__model_meta__']

### Populate the database with `ProteinSequence`s

In [12]:
# Insert all blast results into the database
db.insert(*blast_results, verbose=True)

### Look at entries in the database

In [13]:
db.connection.table("ProteinInfo_organism")

┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ name                    ┃ taxonomy_id ┃ domain    ┃ kingdom ┃ phylum     ┃ tax_class ┃ order   ┃ family        ┃ genus      ┃ species      ┃ ProteinInfo_organism_id              ┃ ProteinInfo_id                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string                  │ !string     │ string    │ string  │ string     │ string    │ string  │ string        │ string     │ string       │ !string                              │ string                               │
├─────────────────────────┼─────────────┼───────────┼─────────┼────────────┼───────────┼─────────┼───────────────┼────────────┼──────────────┼──────────────────────────────────────┼──────────────────────────────────────┤
│ Drosophila melanogaster │ taxon:7227  │ Eukaryota │ Metazoa │ Arthropoda │ Insecta   │ Diptera │ Drosophilidae │ Drosophila │ melanogaster │ 545be718-ed7f-47a3-be8e-5bd7b10ba298 │ 0457be18-0d6f-4f10-a5e6-e5539c32b344 │
│ Drosophila melanogaster │ taxon:7227  │ Eukaryota │ Metazoa │ Arthropoda │ Insecta   │ Diptera │ Drosophilidae │ Drosophila │ melanogaster │ 08242878-164e-40b7-ad53-4384601dbe5b │ 8f5426f4-69e4-46f9-81e8-447b96b6e848 │
│ Drosophila sechellia    │ taxon:7238  │ Eukaryota │ Metazoa │ Arthropoda │ Insecta   │ Diptera │ Drosophilidae │ Drosophila │ sechellia    │ f42d5566-7d14-4158-b8cf-345decd85e67 │ 06d5757e-ba45-40ab-9a81-3f4a434a63bf │
│ Drosophila simulans     │ taxon:7240  │ Eukaryota │ Metazoa │ Arthropoda │ Insecta   │ Diptera │ Drosophilidae │ Drosophila │ simulans     │ af3964d6-8c1a-49d8-b2c9-f6269e307793 │ 1f982911-b07a-4b67-a972-262c85d79c75 │
│ Drosophila melanogaster │ taxon:7227  │ Eukaryota │ Metazoa │ Arthropoda │ Insecta   │ Diptera │ Drosophilidae │ Drosophila │ melanogaster │ 4b64e92f-f1d3-4717-a7d6-d82088dd1dd4 │ 0d04dc96-0f70-4b43-9483-52eef138bfe9 │
│ Drosophila erecta       │ taxon:7220  │ Eukaryota │ Metazoa │ Arthropoda │ Insecta   │ Diptera │ Drosophilidae │ Drosophila │ erecta       │ 9e13b368-ccfd-4669-97fa-ea6f10cc519f │ 70503d04-72fc-4427-bdff-1fe607a3bbe8 │
│ Drosophila yakuba       │ taxon:7245  │ Eukaryota │ Metazoa │ Arthropoda │ Insecta   │ Diptera │ Drosophilidae │ Drosophila │ yakuba       │ eebebdf9-3940-4892-b60b-d4e9463eb993 │ bd13f2d2-9615-436f-9cfd-afb6c453d7fa │
│ Drosophila melanogaster │ taxon:7227  │ Eukaryota │ Metazoa │ Arthropoda │ Insecta   │ Diptera │ Drosophilidae │ Drosophila │ melanogaster │ 4a5ed558-d49c-4ded-ba1e-757a44bdc2b5 │ 6b335d57-be89-4039-8cd4-daccd42b95a1 │
│ Drosophila teissieri    │ taxon:7243  │ Eukaryota │ Metazoa │ Arthropoda │ Insecta   │ Diptera │ Drosophilidae │ Drosophila │ teissieri    │ 870b6c16-3918-4f72-a2c1-96c496c0f1af │ df2bc5c2-4621-4977-b9e9-d24f320eabb6 │
│ Drosophila ficusphila   │ taxon:30025 │ Eukaryota │ Metazoa │ Arthropoda │ Insecta   │ Diptera │ Drosophilidae │ Drosophila │ ficusphila   │ d07123d7-a7fd-43fe-be6f-b1a809a1e7ee │ 39169b24-8215-45b1-bc6b-76d2a7d21bb7 │
│ …                       │ …           │ …         │ …       │ …          │ …         │ …       │ …             │ …          │ …            │ …                                    │ …                                    │
└─────────────────────────┴─────────────┴───────────┴─────────┴────────────┴───────────┴─────────┴───────────────┴────────────┴──────────────┴──────────────────────────────────────┴──────────────────────────────────────┘

In [14]:
# Lets filter the blast results for a specific organism
target = "Drosophila melanogaster"

# First, join the ProteinSequence table with the ProteinSequence_organism table
prot_seqs = db.connection.table("ProteinInfo")
organisms = db.connection.table("ProteinInfo_organism")
joined = prot_seqs.join(
    organisms,
    prot_seqs.ProteinInfo_id == organisms.ProteinInfo_id,
    rname="organism_{name}",
)

# Next, filter the joined table for the target organism
filtered = joined.filter(joined.organism_name == target)
filtered

# Finally, we can get the corresponding ProteinSequence objects
results = db.get("ProteinInfo", filtered)
print(len(results))

10
